# TD SIEM : Suricata, Filebeat, Elasticsearch et Kibana

## Objectifs du TD

Ce TD vous permettra de :
- Configurer Suricata (système de détection d'intrusion) pour générer des logs EVE JSON
- Utiliser Filebeat pour collecter et envoyer automatiquement les logs vers Elasticsearch
- Explorer et analyser les données dans Elasticsearch
- Créer des visualisations dans Kibana
- Configurer des alertes pour la détection de menaces

## Architecture SIEM

```
Suricata → Filebeat → Elasticsearch → Kibana
   (IDS)    (Collecteur)   (Stockage)   (Visualisation/Alerting)
```

## Prérequis

### 1. Installation des dépendances

Assurez-vous d'avoir installé les dépendances du projet :
```bash
uv sync
```

### 2. Préparation des répertoires

Avant de démarrer la stack, créez les répertoires nécessaires pour Suricata :

```bash
mkdir -p suricata/{logs,config,rules}
```

### 3. Configuration de Suricata

Créez le fichier `suricata/config/suricata.yaml` avec la configuration minimale suivante :

```yaml
# Configuration des logs EVE (JSON)
- eve-log:
    enabled: yes
    filetype: regular
    filename: eve.json
    types:
      - alert
      - http
      - dns
      - tls
      - files
      - ssh
      - stats
      - flow
    json: yes

# Interface réseau
af-packet:
  - interface: eth0
    cluster-id: 99
    cluster-type: cluster_flow
    defrag: yes

# Règles
default-rule-path: /var/lib/suricata/rules
rule-files:
  - suricata.rules
```

### 4. Démarrage de la stack complète SIEM

Le fichier `docker-compose-siem.yml` inclut toute la stack nécessaire pour ce TD :
- Cluster Elasticsearch (3 nœuds)
- Kibana
- Suricata (IDS/IPS) - génère des logs dans `./suricata/logs/eve.json`
- Filebeat (collecteur de logs) - lit depuis `./suricata/logs` et indexe dans Elasticsearch

**Démarrage de la stack complète :**
```bash
docker-compose -f docker-compose-siem.yml up -d
```

**Vérification :**
- Elasticsearch : `https://localhost:9200` (ou le port configuré dans `.env`)
- Kibana : `http://localhost:5601` (ou le port configuré dans `.env`)

**Note :** Si vous utilisez un cluster avec sécurité activée, vous devrez utiliser les identifiants définis dans le fichier `.env` (ELASTIC_PASSWORD).

**Important :** Filebeat indexe automatiquement les logs Suricata dans des index nommés `suricata-YYYY.MM.DD`. Les données seront disponibles après que Suricata ait généré des logs et que Filebeat les ait collectés.

### 4. Vérification de la stack

La cellule suivante vérifie que tous les services sont correctement démarrés.

In [5]:
# Vérification de la stack Docker
import subprocess
import os

def check_docker_container(container_name):
    """Vérifie si un conteneur Docker est en cours d'exécution"""
    try:
        result = subprocess.run(
            ['docker', 'ps', '--filter', f'name={container_name}', '--format', '{{.Names}}'],
            capture_output=True,
            text=True,
            timeout=5
        )
        return container_name in result.stdout
    except Exception:
        return False

def check_docker_compose():
    """Vérifie si docker-compose est disponible"""
    try:
        subprocess.run(['docker', 'compose', 'version'], capture_output=True, check=True, timeout=5)
        return True
    except:
        try:
            subprocess.run(['docker-compose', 'version'], capture_output=True, check=True, timeout=5)
            return True
        except:
            return False

# Vérifier Docker
try:
    subprocess.run(['docker', '--version'], capture_output=True, check=True, timeout=5)
    print("✅ Docker est installé")
except (FileNotFoundError, subprocess.CalledProcessError):
    print("❌ Docker n'est pas installé")
    print("   Installez Docker: https://docs.docker.com/get-docker/")
    exit(1)

# Vérifier docker-compose
if check_docker_compose():
    print("✅ Docker Compose est disponible")
else:
    print("❌ Docker Compose n'est pas disponible")

# Vérifier les conteneurs
services = ['es01', 'es02', 'es03', 'kibana', 'suricata', 'filebeat']
print("\n📦 Statut des services:")
for service in services:
    if check_docker_container(service):
        print(f"   ✅ {service}: en cours d'exécution")
    else:
        print(f"   ⚠️  {service}: arrêté")

print("\n📝 Pour arrêter:")
print("   docker-compose down")

✅ Docker est installé
✅ Docker Compose est disponible

📦 Statut des services:
   ✅ es01: en cours d'exécution
   ✅ es02: en cours d'exécution
   ✅ es03: en cours d'exécution
   ✅ kibana: en cours d'exécution
   ✅ suricata: en cours d'exécution
   ✅ filebeat: en cours d'exécution

📝 Pour arrêter:
   docker-compose down


In [6]:
# Import des librairies nécessaires
from elasticsearch import Elasticsearch
from pprint import pprint
import json
import time
from datetime import datetime, timedelta

# Configuration de la connexion Elasticsearch
# Adaptez selon votre configuration (avec ou sans sécurité)
ES_HOST = "https://localhost:9200"
ES_USER = "elastic"  # Modifiez si nécessaire
ES_PASSWORD = "changeme"  # Définissez votre mot de passe si sécurité activée

# Connexion au cluster Elasticsearch
if ES_PASSWORD:
    es = Elasticsearch(
        [ES_HOST],
        basic_auth=(ES_USER, ES_PASSWORD),
        verify_certs=False  # Désactiver la vérification SSL pour le développement
    )
else:
    # Pour un cluster sans sécurité (développement uniquement)
    es = Elasticsearch("http://localhost:9200")

# Vérification de la connexion
try:
    health = es.cluster.health()
    print("✅ Connexion établie avec Elasticsearch")
    print(f"   Statut du cluster: {health['status']}")
    print(f"   Nombre de nœuds: {health['number_of_nodes']}")
except Exception as e:
    print(f"❌ Erreur de connexion: {e}")
    print("   Vérifiez que votre cluster Elasticsearch est démarré")

✅ Connexion établie avec Elasticsearch
   Statut du cluster: green
   Nombre de nœuds: 3


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/elasticsearch/_sync/client/__init__.py:313: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Partie 1 : Configuration de Suricata

### 1.1 Vérification de l'installation de Suricata

In [7]:
# Création des répertoires nécessaires pour Suricata
import os

# Créer les répertoires si nécessaire
directories = [
    './suricata/logs',
    './suricata/config',
    './suricata/rules',
    './suricata_logs'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"✅ Répertoire créé/vérifié: {directory}")

print("\n📝 Les répertoires sont prêts pour Suricata")

✅ Répertoire créé/vérifié: ./suricata/logs
✅ Répertoire créé/vérifié: ./suricata/config
✅ Répertoire créé/vérifié: ./suricata/rules
✅ Répertoire créé/vérifié: ./suricata_logs

📝 Les répertoires sont prêts pour Suricata


### 1.2 Vérification des logs Suricata

Suricata génère des logs au format JSON (EVE logs) dans `./suricata/logs/eve.json`. 
Filebeat lit automatiquement ces logs et les indexe dans Elasticsearch.

Vérifions que les logs sont générés et que Filebeat les collecte correctement.

In [28]:
# Vérification des logs Suricata et de Filebeat
import subprocess
import time

# Vérifier que le fichier de logs existe
log_file = "./suricata/logs/eve.json"
if os.path.exists(log_file):
    # Compter le nombre de lignes (logs)
    with open(log_file, 'r') as f:
        log_count = sum(1 for line in f if line.strip())
    print(f"✅ Fichier de logs trouvé: {log_file}")
    print(f"   Nombre de lignes (logs): {log_count}")
    
    # Afficher les dernières lignes
    if log_count > 0:
        print("\n📄 Derniers logs (3 dernières lignes):")
        with open(log_file, 'r') as f:
            lines = f.readlines()
            for line in lines[-3:]:
                try:
                    log_entry = json.loads(line.strip())
                    print(f"   - {log_entry.get('event_type', 'unknown')} à {log_entry.get('timestamp', 'N/A')}")
                except:
                    print(f"   - (ligne non-JSON)")
else:
    print(f"⚠️  Fichier de logs non trouvé: {log_file}")
    print("   Assurez-vous que Suricata est démarré et génère des logs")
    print("   Les logs seront créés automatiquement quand Suricata détecte du trafic")

# Vérifier le statut de Filebeat
print("\n🔍 Vérification de Filebeat:")
try:
    result = subprocess.run(
        ['docker', 'logs', '--tail', '10', 'filebeat'],
        capture_output=True,
        text=True,
        timeout=5
    )
    if result.returncode == 0:
        print("✅ Filebeat est en cours d'exécution")
        # Chercher des erreurs dans les logs
        if 'error' in result.stdout.lower() or 'error' in result.stderr.lower():
            print("   ⚠️  Des erreurs détectées dans les logs Filebeat")
        else:
            print("   ✅ Aucune erreur détectée")
    else:
        print("   ⚠️  Impossible de récupérer les logs Filebeat")
except Exception as e:
    print(f"   ⚠️  Erreur lors de la vérification: {e}")

✅ Fichier de logs trouvé: ./suricata/logs/eve.json
   Nombre de lignes (logs): 64741

📄 Derniers logs (3 dernières lignes):
   - flow à 2026-01-17T16:33:56.263903+0000
   - flow à 2026-01-17T16:33:56.263916+0000
   - stats à 2026-01-17T16:33:56.656711+0000

🔍 Vérification de Filebeat:
✅ Filebeat est en cours d'exécution
   ⚠️  Des erreurs détectées dans les logs Filebeat


In [12]:
# Vérification des index créés par Filebeat
# Filebeat 9.x crée des data streams avec le pattern: .ds-suricata-YYYY.MM.DD-*
# ou des index normaux avec le pattern: suricata-YYYY.MM.DD

from datetime import datetime

# Obtenir la date actuelle pour construire le nom d'index
today = datetime.now().strftime("%Y.%m.%d")

print(f"🔍 Recherche des index Suricata créés par Filebeat...")
print(f"   Patterns attendus: suricata-* ou .ds-suricata-*")
print(f"   Date du jour: {today}\n")

# Lister tous les index qui correspondent aux patterns
all_indices = []
try:
    # Chercher les index normaux (utiliser get qui fonctionne mieux)
    try:
        indices_normal = es.indices.get(index="suricata-*")
        if indices_normal:
            all_indices.extend(indices_normal.keys())
    except:
        # Fallback sur get_alias si get échoue
        try:
            indices_normal = es.indices.get_alias(index="suricata-*")
            if indices_normal:
                all_indices.extend(indices_normal.keys())
        except:
            pass
    
    # Chercher les data streams (commencent par .ds-) - get_alias ne fonctionne pas avec les data streams
    try:
        indices_ds = es.indices.get(index=".ds-suricata-*")
        if indices_ds:
            all_indices.extend(indices_ds.keys())
    except:
        pass
    
    if all_indices:
        print(f"✅ {len(all_indices)} index(s) trouvé(s):")
        for index_name in sorted(all_indices):
            try:
                count = es.count(index=index_name)
                print(f"   - {index_name}: {count['count']} documents")
            except:
                print(f"   - {index_name}: (erreur lors du comptage)")
    else:
        print("⚠️  Aucun index trouvé avec les patterns 'suricata-*' ou '.ds-suricata-*'")
        print("   Cela peut signifier que:")
        print("   1. Filebeat n'a pas encore indexé de données")
        print("   2. Suricata n'a pas encore généré de logs")
        print("   3. Il y a un problème de configuration")
        print("   Vérifiez les logs Filebeat: docker logs filebeat")
except Exception as e:
    print(f"⚠️  Erreur lors de la recherche des index: {e}")
    print("   Vérifiez que Elasticsearch est accessible et que Filebeat fonctionne")

🔍 Recherche des index Suricata créés par Filebeat...
   Patterns attendus: suricata-* ou .ds-suricata-*
   Date du jour: 2026.01.17

✅ 2 index(s) trouvé(s):
   - .ds-suricata-2026.01.17-2026.01.17-000001: 121272 documents
   - .ds-suricata-2026.01.17-2026.01.17-000001: 121272 documents


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanc

## Partie 2 : Utilisation des données indexées par Filebeat

Filebeat indexe automatiquement les logs Suricata dans des index nommés `suricata-YYYY.MM.DD`. 
Nous allons maintenant explorer ces données dans Elasticsearch.

**Note importante :** Les données doivent être indexées par Filebeat. Si vous ne voyez pas d'index, 
attendez quelques instants que Suricata génère des logs et que Filebeat les collecte.

In [29]:
# Fonction utilitaire pour obtenir le nom de l'index à utiliser
def get_suricata_index():
    """Retourne le nom de l'index Suricata le plus récent ou un pattern"""
    try:
        all_indices = []
        # Chercher les index normaux (utiliser get qui fonctionne mieux)
        try:
            indices_normal = es.indices.get(index="suricata-*")
            if indices_normal:
                all_indices.extend(indices_normal.keys())
        except:
            # Fallback sur get_alias si get échoue
            try:
                indices_normal = es.indices.get_alias(index="suricata-*")
                if indices_normal:
                    all_indices.extend(indices_normal.keys())
            except:
                pass
        
        # Chercher les data streams (commencent par .ds-) - get_alias ne fonctionne pas avec les data streams
        try:
            indices_ds = es.indices.get(index=".ds-suricata-*")
            if indices_ds:
                all_indices.extend(indices_ds.keys())
        except:
            pass
        
        if all_indices:
            # Retourner l'index le plus récent
            return sorted(all_indices)[-1]
        else:
            # Si aucun index n'existe, utiliser le pattern
            return "suricata-*"
    except:
        # En cas d'erreur, utiliser le pattern
        return "suricata-*"

# Fonction pour obtenir l'index de recherche (utilise le pattern si l'index spécifique n'existe pas)
def get_search_index():
    """Retourne l'index à utiliser pour les recherches (inclut les data streams)"""
    index_name = get_suricata_index()
    if index_name == "suricata-*":
        # Utiliser les deux patterns pour couvrir index normaux et data streams
        return "suricata-*,.ds-suricata-*"
    # Vérifier si l'index existe
    try:
        if es.indices.exists(index=index_name):
            return index_name
        else:
            return "suricata-*,.ds-suricata-*"
    except:
        return "suricata-*,.ds-suricata-*"

# Déterminer l'index à utiliser
index_name = get_suricata_index()
print(f"📊 Index détecté: {index_name}")

# Vérifier si des index existent (chercher les deux types)
try:
    all_indices_found = []
    
    # Chercher les index normaux
    try:
        indices_normal = es.indices.get(index="suricata-*")
        if indices_normal:
            all_indices_found.extend(indices_normal.keys())
    except:
        pass
    
    # Chercher les data streams
    try:
        indices_ds = es.indices.get(index=".ds-suricata-*")
        if indices_ds:
            all_indices_found.extend(indices_ds.keys())
    except:
        pass
    
    if all_indices_found:
        print(f"✅ {len(all_indices_found)} index(s) trouvé(s):")
        for idx in sorted(all_indices_found):
            try:
                count = es.count(index=idx)
                print(f"   - {idx}: {count['count']} document(s)")
            except:
                print(f"   - {idx}: (erreur lors du comptage)")
    else:
        print("⚠️  Aucun index trouvé avec les patterns 'suricata-*' ou '.ds-suricata-*'")
        print("   Filebeat créera des index automatiquement lors de la première indexation")
        print("   Assurez-vous que:")
        print("   1. Suricata génère des logs dans ./suricata/logs/eve.json")
        print("   2. Filebeat est démarré et fonctionne correctement")
except Exception as e:
    print(f"⚠️  Erreur lors de la vérification des index: {e}")

📊 Index détecté: .ds-suricata-2026.01.17-2026.01.17-000001
✅ 2 index(s) trouvé(s):
   - .ds-suricata-2026.01.17-2026.01.17-000001: 124107 document(s)
   - .ds-suricata-2026.01.17-2026.01.17-000001: 124107 document(s)


In [35]:
# affiche le nombre de documents dans l'index
es.count(index=index_name)

KeyboardInterrupt: 

### 2.1 Exploration des données indexées

Les données sont déjà indexées par Filebeat. Explorons-les maintenant.

In [14]:
# Recherche de tous les logs indexés par Filebeat
# Utiliser le pattern suricata-* pour rechercher dans tous les index

try:
    # Vérifier d'abord si l'index existe
    if not es.indices.exists(index=index_name):
        print(f"⚠️  L'index {index_name} n'existe pas encore.")
        print("   Attendez que Filebeat indexe les logs de Suricata.")
        print("   Vous pouvez vérifier les logs Filebeat avec: docker logs filebeat")
    else:
        # Recherche de base
        query_all = {
            "match_all": {}
        }
        
        result = es.search(index=index_name, query=query_all, size=10)
        total = result['hits']['total']['value']
        
        print(f"📊 Total de documents dans l'index '{index_name}': {total}")
        print(f"   Documents retournés: {len(result['hits']['hits'])}\n")
        
        if total > 0:
            print("📄 Exemples de documents indexés:\n")
            for i, hit in enumerate(result['hits']['hits'][:5], 1):
                source = hit['_source']
                print(f"Document {i}:")
                print(f"   Type d'événement: {source.get('event_type', 'N/A')}")
                if 'src_ip' in source:
                    print(f"   Source: {source.get('src_ip', 'N/A')}:{source.get('src_port', 'N/A')}")
                    print(f"   Destination: {source.get('dest_ip', 'N/A')}:{source.get('dest_port', 'N/A')}")
                if 'alert' in source:
                    alert = source['alert']
                    print(f"   Alerte: {alert.get('signature', 'N/A')}")
                    print(f"   Sévérité: {alert.get('severity', 'N/A')}")
                if '@timestamp' in source:
                    print(f"   Timestamp: {source.get('@timestamp', 'N/A')}")
                print()
        else:
            print("⚠️  L'index existe mais ne contient pas encore de documents.")
            print("   Vérifiez que Suricata génère des logs et que Filebeat les collecte.")
            
except Exception as e:
    print(f"❌ Erreur lors de la recherche: {e}")
    print("   Vérifiez que Elasticsearch est accessible et que l'index existe.")

📊 Total de documents dans l'index '.ds-suricata-2026.01.17-2026.01.17-000001': 10000
   Documents retournés: 10

📄 Exemples de documents indexés:

Document 1:
   Type d'événement: flow
   Source: 192.168.65.1:41588
   Destination: 192.168.65.7:2376
   Timestamp: 2026-01-17T16:31:15.700Z

Document 2:
   Type d'événement: flow
   Source: 192.168.65.1:21106
   Destination: 192.168.65.7:2376
   Timestamp: 2026-01-17T16:31:15.700Z

Document 3:
   Type d'événement: flow
   Source: 192.168.65.1:33296
   Destination: 192.168.65.7:2376
   Timestamp: 2026-01-17T16:31:15.700Z

Document 4:
   Type d'événement: flow
   Source: 192.168.65.1:63138
   Destination: 192.168.65.7:2376
   Timestamp: 2026-01-17T16:31:15.700Z

Document 5:
   Type d'événement: flow
   Source: 192.168.65.1:31983
   Destination: 192.168.65.7:2376
   Timestamp: 2026-01-17T16:31:15.700Z



/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Partie 3 : Exploration des données dans Elasticsearch

### 3.1 Recherche de base dans les logs Suricata

In [17]:
# Recherche de tous les logs
query_all = {
    "match_all": {}
}

result = es.search(index=index_name, query=query_all, size=10)
print(f"📊 Total de documents: {result['hits']['total']['value']}")
print(f"   Documents retournés: {len(result['hits']['hits'])}\n")

for hit in result['hits']['hits']:
    source = hit['_source']
    print(f"📄 Document {hit['_id']}:")
    print(f"   Type: {source.get('event_type', 'N/A')}")
    if 'alert' in source:
        print(f"   Alerte: {source['alert'].get('signature', 'N/A')}")
    print(f"   Source IP: {source.get('src_ip', 'N/A')} → Dest IP: {source.get('dest_ip', 'N/A')}")
    print()

📊 Total de documents: 10000
   Documents retournés: 10

📄 Document VRfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document VhfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document VxfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document WBfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document WRfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document WhfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document WxfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document XBfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document XRfMzJsBtz7UXYiikMD4:
   Type: flow
   Source IP: 192.168.65.1 → Dest IP: 192.168.65.7

📄 Document XhfMzJsBtz7UXYiikMD4:
   Type: flow
   Sou

/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 3.2 Recherche d'alertes spécifiques

In [18]:
# Rechercher toutes les alertes (event_type = "alert")
query_alerts = {
    "bool": {
        "must": [
            {"term": {"event_type": "alert"}}
        ]
    }
}

result = es.search(index=index_name, query=query_alerts, size=100)
print(f"🚨 Nombre d'alertes trouvées: {result['hits']['total']['value']}\n")

for hit in result['hits']['hits']:
    source = hit['_source']
    alert = source.get('alert', {})
    print(f"⚠️  Alerte ID: {alert.get('signature_id', 'N/A')}")
    print(f"   Signature: {alert.get('signature', 'N/A')}")
    print(f"   Catégorie: {alert.get('category', 'N/A')}")
    print(f"   Sévérité: {alert.get('severity', 'N/A')}")
    print(f"   Source: {source.get('src_ip', 'N/A')}:{source.get('src_port', 'N/A')}")
    print(f"   Destination: {source.get('dest_ip', 'N/A')}:{source.get('dest_port', 'N/A')}")
    print()

🚨 Nombre d'alertes trouvées: 0



/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 3.3 Recherche d'alertes de haute sévérité

In [19]:
# Rechercher les alertes de haute sévérité (severity <= 1)
query_high_severity = {
    "bool": {
        "must": [
            {"term": {"event_type": "alert"}},
            {"range": {"alert.severity": {"lte": 1}}}
        ]
    }
}

result = es.search(index=index_name, query=query_high_severity, size=100)
print(f"🔴 Alertes de haute sévérité (≤1): {result['hits']['total']['value']}\n")

for hit in result['hits']['hits']:
    source = hit['_source']
    alert = source.get('alert', {})
    print(f"🔴 CRITIQUE - Sévérité {alert.get('severity', 'N/A')}")
    print(f"   {alert.get('signature', 'N/A')}")
    print(f"   Source IP suspecte: {source.get('src_ip', 'N/A')}")
    print()

🔴 Alertes de haute sévérité (≤1): 0



/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


### 3.4 Agrégations : Statistiques sur les alertes

In [20]:
# Agrégations pour analyser les données
aggs_query = {
    "size": 0,
    "aggs": {
        "alertes_par_categorie": {
            "terms": {
                "field": "alert.category",
                "size": 10
            }
        },
        "alertes_par_severite": {
            "terms": {
                "field": "alert.severity",
                "size": 5
            }
        },
        "top_source_ips": {
            "terms": {
                "field": "src_ip",
                "size": 10
            }
        },
        "top_dest_ips": {
            "terms": {
                "field": "dest_ip",
                "size": 10
            }
        },
        "top_signatures": {
            "terms": {
                "field": "alert.signature.keyword",
                "size": 10
            }
        }
    }
}

result = es.search(index=index_name, body=aggs_query)

print("📊 Statistiques sur les alertes\n")
print("=" * 50)

# Alertes par catégorie
if 'alertes_par_categorie' in result['aggregations']:
    print("\n📁 Alertes par catégorie:")
    for bucket in result['aggregations']['alertes_par_categorie']['buckets']:
        print(f"   {bucket['key']}: {bucket['doc_count']}")

# Alertes par sévérité
if 'alertes_par_severite' in result['aggregations']:
    print("\n⚠️  Alertes par sévérité:")
    for bucket in result['aggregations']['alertes_par_severite']['buckets']:
        print(f"   Sévérité {bucket['key']}: {bucket['doc_count']}")

# Top IPs sources
if 'top_source_ips' in result['aggregations']:
    print("\n🌐 Top 10 IPs sources:")
    for bucket in result['aggregations']['top_source_ips']['buckets']:
        print(f"   {bucket['key']}: {bucket['doc_count']} événements")

# Top signatures
if 'top_signatures' in result['aggregations']:
    print("\n🔍 Top 10 signatures:")
    for bucket in result['aggregations']['top_signatures']['buckets']:
        print(f"   {bucket['key']}: {bucket['doc_count']}")

📊 Statistiques sur les alertes


📁 Alertes par catégorie:

⚠️  Alertes par sévérité:

🌐 Top 10 IPs sources:
   192.168.65.1: 80907 événements
   192.168.65.7: 40147 événements
   192.168.65.3: 427 événements

🔍 Top 10 signatures:


/home/mp/code/formations/elasticsearch_cookbook/.venv/lib/python3.14/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Partie 4 : Visualisation dans Kibana

### 4.1 Accès à Kibana

1. Ouvrez votre navigateur et allez sur `http://localhost:5601`
2. Connectez-vous avec les identifiants définis dans votre fichier `.env` (si sécurité activée)
3. Si c'est la première fois, vous devrez peut-être accepter les termes de licence

### 4.2 Création d'un index pattern

Pour visualiser les données dans Kibana, vous devez créer un **index pattern** :

1. Allez dans **Management** → **Stack Management** → **Index Patterns**
2. Cliquez sur **Create index pattern**
3. Entrez le pattern : `suricata-*` ou `.ds-suricata-*` (selon la version de Filebeat)
   - Filebeat 9.x crée des **data streams** avec le pattern `.ds-suricata-*`
   - Les versions antérieures créent des index normaux avec `suricata-*`
4. Sélectionnez le champ de temps : `@timestamp` (ajouté par Filebeat)
5. Cliquez sur **Create index pattern**

**Note :** 
- Filebeat 9.x utilise des data streams (recommandé par Elastic)
- Les data streams sont automatiquement gérés et optimisés par Elasticsearch
- Utilisez `suricata-*` ou `.ds-suricata-*` selon ce que Filebeat a créé

### 4.3 Création de visualisations

Dans Kibana, créez les visualisations suivantes :

#### Visualisation 1 : Timeline des alertes
- **Type** : Line chart ou Area chart
- **Métrique** : Count
- **Buckets** : Date Histogram sur `@timestamp`
- **Filtre** : `event_type: alert`

#### Visualisation 2 : Top 10 signatures d'alertes
- **Type** : Horizontal Bar chart
- **Métrique** : Count
- **Buckets** : Terms sur `alert.signature.keyword` (Top 10)

#### Visualisation 3 : Répartition par sévérité
- **Type** : Pie chart
- **Métrique** : Count
- **Buckets** : Terms sur `alert.severity`

#### Visualisation 4 : Carte des IPs sources
- **Type** : Coordinate Map ou Tile Map
- **Métrique** : Count
- **Buckets** : Geohash sur `src_ip` (si géolocalisation activée)

### 4.4 Création d'un Dashboard

1. Allez dans **Dashboard** → **Create dashboard**
2. Ajoutez les visualisations créées précédemment
3. Configurez les filtres temporels
4. Sauvegardez le dashboard

**Note :** Pour ce TD, nous allons également créer des visualisations programmatiquement via l'API.

In [21]:
# Vérification de l'accès à Kibana (via l'API REST)
import requests
import urllib3

# Désactiver les avertissements SSL pour le développement
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

KIBANA_URL = "http://localhost:5601"
KIBANA_USER = "elastic"  # Modifiez si nécessaire
KIBANA_PASSWORD = None  # Définissez votre mot de passe si sécurité activée

# Test de connexion à Kibana
try:
    if KIBANA_PASSWORD:
        response = requests.get(
            f"{KIBANA_URL}/api/status",
            auth=(KIBANA_USER, KIBANA_PASSWORD),
            verify=False,
            timeout=5
        )
    else:
        response = requests.get(f"{KIBANA_URL}/api/status", timeout=5)
    
    if response.status_code == 200:
        print("✅ Kibana est accessible")
        status = response.json()
        print(f"   Version: {status.get('version', {}).get('number', 'N/A')}")
    else:
        print(f"⚠️  Kibana répond mais avec le code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Impossible de se connecter à Kibana")
    print("   Vérifiez que Kibana est démarré sur http://localhost:5601")
except Exception as e:
    print(f"⚠️  Erreur: {e}")

✅ Kibana est accessible
   Version: N/A


## Partie 5 : Alerting dans Kibana

### 5.1 Création de règles d'alerte

Kibana permet de créer des règles d'alerte basées sur des requêtes Elasticsearch. Voici quelques exemples de règles utiles pour un SIEM :

#### Règle 1 : Alerte sur haute sévérité
- **Condition** : Nombre d'alertes avec `severity <= 1` > 0
- **Action** : Envoyer une notification (email, Slack, etc.)

#### Règle 2 : Alerte sur scan de ports
- **Condition** : Plus de 10 alertes de type "SCAN" en 5 minutes
- **Action** : Notification immédiate

#### Règle 3 : Alerte sur IP suspecte
- **Condition** : Plus de 5 alertes depuis la même IP source en 10 minutes
- **Action** : Bloquer l'IP (si intégration avec firewall)

### 5.2 Configuration des alertes via l'API

Pour créer des alertes programmatiquement, utilisez l'API Kibana Alerting.

In [22]:
# Exemple de création d'une règle d'alerte Kibana
# Note: Cette fonctionnalité nécessite Kibana avec les fonctionnalités d'alerting activées

def create_alert_rule(rule_name, query, threshold, time_window="5m"):
    """
    Crée une règle d'alerte Kibana
    
    Args:
        rule_name: Nom de la règle
        query: Requête Elasticsearch (dict)
        threshold: Seuil d'alerte
        time_window: Fenêtre temporelle (ex: "5m", "1h")
    """
    alert_rule = {
        "name": rule_name,
        "consumer": "alerts",
        "enabled": True,
        "rule_type_id": ".es-query",
        "schedule": {
            "interval": "1m"
        },
        "actions": [
            {
                "group": "query matched",
                "id": "my-connector-id",  # À remplacer par un ID de connecteur réel
                "params": {
                    "message": f"Alerte: {rule_name} - Seuil dépassé!"
                }
            }
        ],
        "params": {
            "index": ["suricata-*"],
            "query": query,
            "size": 100,
            "timeField": "@timestamp",
            "timeWindowSize": time_window,
            "threshold": [
                {
                    "field": "count",
                    "value": threshold,
                    "comparator": ">"
                }
            ]
        }
    }
    
    return alert_rule

# Exemple: Règle d'alerte pour haute sévérité
high_severity_query = {
    "bool": {
        "must": [
            {"term": {"event_type": "alert"}},
            {"range": {"alert.severity": {"lte": 1}}}
        ]
    }
}

alert_rule = create_alert_rule(
    rule_name="Alerte haute sévérité Suricata",
    query=high_severity_query,
    threshold=0,
    time_window="5m"
)

print("📋 Exemple de règle d'alerte créée:")
print(json.dumps(alert_rule, indent=2))

print("\n💡 Pour créer cette règle dans Kibana:")
print("   1. Allez dans Stack Management → Rules and Connectors")
print("   2. Créez un connecteur (email, Slack, etc.)")
print("   3. Créez une nouvelle règle avec les paramètres ci-dessus")

📋 Exemple de règle d'alerte créée:
{
  "name": "Alerte haute s\u00e9v\u00e9rit\u00e9 Suricata",
  "consumer": "alerts",
  "enabled": true,
  "rule_type_id": ".es-query",
  "schedule": {
    "interval": "1m"
  },
  "actions": [
    {
      "group": "query matched",
      "id": "my-connector-id",
      "params": {
        "message": "Alerte: Alerte haute s\u00e9v\u00e9rit\u00e9 Suricata - Seuil d\u00e9pass\u00e9!"
      }
    }
  ],
  "params": {
    "index": [
      "suricata-*"
    ],
    "query": {
      "bool": {
        "must": [
          {
            "term": {
              "event_type": "alert"
            }
          },
          {
            "range": {
              "alert.severity": {
                "lte": 1
              }
            }
          }
        ]
      }
    },
    "size": 100,
    "timeField": "@timestamp",
    "timeWindowSize": "5m",
    "threshold": [
      {
        "field": "count",
        "value": 0,
        "comparator": ">"
      }
    ]
  }
}

💡 Pour 

In [23]:
# Détection de patterns suspects

# 1. Détection d'IP source avec beaucoup d'alertes (possible scan/attaque)
def detect_suspicious_source_ips(min_alerts=3):
    """Détecte les IPs sources avec un nombre élevé d'alertes"""
    query = {
        "size": 0,
        "query": {
            "term": {"event_type": "alert"}
        },
        "aggs": {
            "suspicious_ips": {
                "terms": {
                    "field": "src_ip",
                    "size": 20,
                    "min_doc_count": min_alerts
                },
                "aggs": {
                    "alert_details": {
                        "top_hits": {
                            "size": 5,
                            "_source": {
                                "includes": ["alert.signature", "alert.severity", "dest_ip", "dest_port"]
                            }
                        }
                    }
                }
            }
        }
    }
    
    result = es.search(index=index_name, body=query)
    
    print(f"🔍 IPs sources suspectes (≥{min_alerts} alertes):\n")
    for bucket in result['aggregations']['suspicious_ips']['buckets']:
        ip = bucket['key']
        count = bucket['doc_count']
        print(f"⚠️  {ip}: {count} alertes")
        
        # Afficher les détails des alertes
        for hit in bucket['alert_details']['hits']['hits']:
            source = hit['_source']
            alert = source.get('alert', {})
            print(f"   - {alert.get('signature', 'N/A')} (Sévérité: {alert.get('severity', 'N/A')})")
        print()
    
    return result

# 2. Détection d'alertes récentes (dernières 30 minutes)
def detect_recent_alerts(minutes=30):
    """Détecte les alertes récentes"""
    time_threshold = datetime.now() - timedelta(minutes=minutes)
    
    query = {
        "bool": {
            "must": [
                {"term": {"event_type": "alert"}}
            ],
            "filter": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": time_threshold.isoformat()
                        }
                    }
                }
            ]
        }
    }
    
    result = es.search(index=index_name, query=query, size=100, sort=[{"@timestamp": {"order": "desc"}}])
    
    print(f"🕐 Alertes des dernières {minutes} minutes: {result['hits']['total']['value']}\n")
    
    for hit in result['hits']['hits']:
        source = hit['_source']
        alert = source.get('alert', {})
        timestamp = source.get('@timestamp', source.get('timestamp', 'N/A'))
        print(f"⏰ {timestamp}")
        print(f"   {alert.get('signature', 'N/A')}")
        print(f"   {source.get('src_ip', 'N/A')} → {source.get('dest_ip', 'N/A')}")
        print()
    
    return result

# Exécuter les détections
print("=" * 60)
detect_suspicious_source_ips(min_alerts=1)
print("\n" + "=" * 60)
detect_recent_alerts(minutes=60)

🔍 IPs sources suspectes (≥1 alertes):


🕐 Alertes des dernières 60 minutes: 0



ObjectApiResponse({'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}})

In [24]:
# Exemple de requête pour analyser les patterns temporels (utile pour ML)
# Cette requête peut être utilisée comme base pour un job ML

def analyze_temporal_patterns():
    """Analyse les patterns temporels dans les alertes"""
    
    # Agrégation par heure de la journée
    query = {
        "size": 0,
        "query": {
            "term": {"event_type": "alert"}
        },
        "aggs": {
            "alerts_by_hour": {
                "date_histogram": {
                    "field": "@timestamp",
                    "calendar_interval": "hour",
                    "min_doc_count": 1
                },
                "aggs": {
                    "by_severity": {
                        "terms": {
                            "field": "alert.severity",
                            "size": 5
                        }
                    },
                    "unique_ips": {
                        "cardinality": {
                            "field": "src_ip"
                        }
                    }
                }
            },
            "alerts_by_day": {
                "date_histogram": {
                    "field": "@timestamp",
                    "calendar_interval": "day",
                    "min_doc_count": 1
                }
            }
        }
    }
    
    result = es.search(index=index_name, body=query)
    
    print("📈 Analyse des patterns temporels\n")
    
    # Afficher les alertes par heure
    if 'alerts_by_hour' in result['aggregations']:
        print("🕐 Distribution des alertes par heure:")
        for bucket in result['aggregations']['alerts_by_hour']['buckets']:
            time_str = bucket['key_as_string']
            count = bucket['doc_count']
            unique_ips = bucket['unique_ips']['value']
            print(f"   {time_str}: {count} alertes ({unique_ips} IPs uniques)")
    
    print(f"\n📊 Total d'alertes analysées: {result['hits']['total']['value']}")
    
    return result

# Exécuter l'analyse
try:
    analyze_temporal_patterns()
except Exception as e:
    print(f"⚠️  Erreur lors de l'analyse: {e}")
    print("   Assurez-vous que les données contiennent des timestamps valides")

📈 Analyse des patterns temporels

🕐 Distribution des alertes par heure:

📊 Total d'alertes analysées: 0


### 6.4 Détection d'anomalies basique (sans ML)

Même sans activer ML, nous pouvons créer des règles de détection d'anomalies basiques.

In [25]:
# Détection d'anomalies basique : identifier les comportements inhabituels

def detect_anomalies():
    """Détecte des anomalies basiques dans les logs"""
    
    anomalies = []
    
    # 1. IP source avec trop d'alertes différentes (possible botnet/scan)
    query1 = {
        "size": 0,
        "query": {"term": {"event_type": "alert"}},
        "aggs": {
            "ips_with_many_alerts": {
                "terms": {
                    "field": "src_ip",
                    "size": 10
                },
                "aggs": {
                    "unique_signatures": {
                        "cardinality": {
                            "field": "alert.signature_id"
                        }
                    },
                    "unique_dest_ips": {
                        "cardinality": {
                            "field": "dest_ip"
                        }
                    }
                }
            }
        }
    }
    
    result1 = es.search(index=index_name, body=query1)
    
    print("🔍 Détection d'anomalies basiques\n")
    print("=" * 60)
    
    # Analyser les résultats
    for bucket in result1['aggregations']['ips_with_many_alerts']['buckets']:
        ip = bucket['key']
        total_alerts = bucket['doc_count']
        unique_signatures = bucket['unique_signatures']['value']
        unique_dest_ips = bucket['unique_dest_ips']['value']
        
        # Critères d'anomalie
        if unique_signatures > 3 or unique_dest_ips > 5:
            anomaly = {
                "type": "Suspicious scanning activity",
                "ip": ip,
                "total_alerts": total_alerts,
                "unique_signatures": unique_signatures,
                "unique_dest_ips": unique_dest_ips
            }
            anomalies.append(anomaly)
            
            print(f"🚨 ANOMALIE DÉTECTÉE:")
            print(f"   IP: {ip}")
            print(f"   Type: Activité de scan suspecte")
            print(f"   Raison: {unique_signatures} signatures différentes, {unique_dest_ips} IPs de destination différentes")
            print(f"   Total alertes: {total_alerts}")
            print()
    
    # 2. Alertes de haute sévérité récentes
    query2 = {
        "bool": {
            "must": [
                {"term": {"event_type": "alert"}},
                {"range": {"alert.severity": {"lte": 1}}}
            ],
            "filter": [
                {
                    "range": {
                        "@timestamp": {
                            "gte": (datetime.now() - timedelta(hours=1)).isoformat()
                        }
                    }
                }
            ]
        }
    }
    
    result2 = es.search(index=index_name, query=query2, size=100)
    
    if result2['hits']['total']['value'] > 0:
        print("🚨 ALERTES CRITIQUES RÉCENTES:")
        for hit in result2['hits']['hits']:
            source = hit['_source']
            alert = source.get('alert', {})
            print(f"   - {alert.get('signature', 'N/A')}")
            print(f"     Source: {source.get('src_ip', 'N/A')}")
            print(f"     Sévérité: {alert.get('severity', 'N/A')}")
        print()
    
    return anomalies

# Exécuter la détection
anomalies = detect_anomalies()

if not anomalies:
    print("✅ Aucune anomalie majeure détectée avec les critères actuels")

🔍 Détection d'anomalies basiques

✅ Aucune anomalie majeure détectée avec les critères actuels


## Partie 7 : Exercices pratiques

### Exercice 1 : Recherche d'IPs malveillantes

Créez une requête pour trouver toutes les alertes provenant d'une IP spécifique et analysez les signatures déclenchées.

### Exercice 2 : Analyse de trafic HTTP

Recherchez tous les événements HTTP et identifiez les requêtes suspectes (ex: tentatives d'accès à `/admin`, `/wp-admin`, etc.).

### Exercice 3 : Dashboard personnalisé

Créez un dashboard Kibana avec :
- Timeline des alertes
- Top 10 des signatures
- Carte des IPs sources
- Graphique de sévérité

### Exercice 4 : Règle d'alerte personnalisée

Créez une règle d'alerte qui se déclenche lorsqu'une IP génère plus de 5 alertes en 10 minutes.

### Exercice 5 : Détection de patterns

Écrivez une requête pour détecter les scans de ports (plusieurs tentatives de connexion sur différents ports depuis la même IP).

In [26]:
# EXERCICE 1 : Recherche d'IPs malveillantes
# Complétez cette fonction pour rechercher toutes les alertes d'une IP spécifique

def search_alerts_by_ip(ip_address):
    """
    Recherche toutes les alertes pour une IP source donnée
    
    Args:
        ip_address: Adresse IP à rechercher (ex: "192.168.1.100")
    
    Returns:
        Résultats de la recherche
    """
    # TODO: Créer la requête Elasticsearch
    query = {
        "bool": {
            "must": [
                {"term": {"event_type": "alert"}},
                {"term": {"src_ip": ip_address}}
            ]
        }
    }
    
    result = es.search(index=index_name, query=query, size=100)
    
    print(f"🔍 Alertes pour l'IP {ip_address}: {result['hits']['total']['value']}\n")
    
    signatures = {}
    for hit in result['hits']['hits']:
        source = hit['_source']
        alert = source.get('alert', {})
        sig_id = alert.get('signature_id', 'N/A')
        sig_name = alert.get('signature', 'N/A')
        
        if sig_id not in signatures:
            signatures[sig_id] = {
                "name": sig_name,
                "count": 0,
                "severity": alert.get('severity', 'N/A')
            }
        signatures[sig_id]["count"] += 1
    
    print("📋 Signatures déclenchées:")
    for sig_id, info in signatures.items():
        print(f"   [{sig_id}] {info['name']}: {info['count']} fois (Sévérité: {info['severity']})")
    
    return result

# Test avec une IP de test
# Récupérer d'abord quelques alertes pour obtenir une IP de test
test_query = {
    "bool": {
        "must": [
            {"term": {"event_type": "alert"}}
        ]
    }
}
test_result = es.search(index=index_name, query=test_query, size=1)

if test_result['hits']['total']['value'] > 0:
    test_ip = test_result['hits']['hits'][0]['_source'].get('src_ip')
    if test_ip:
        print(f"\n🧪 Test avec l'IP: {test_ip}\n")
        search_alerts_by_ip(test_ip)
    else:
        print("⚠️  Aucune IP source trouvée dans les alertes")
else:
    print("⚠️  Aucune alerte trouvée pour le test")

⚠️  Aucune alerte trouvée pour le test


In [27]:
# EXERCICE 2 : Analyse de trafic HTTP suspect
# Recherchez les requêtes HTTP suspectes

def find_suspicious_http_requests():
    """Recherche les requêtes HTTP suspectes"""
    
    # URLs suspectes communes
    suspicious_paths = ["/admin", "/wp-admin", "/phpmyadmin", "/.env", "/config.php"]
    
    query = {
        "bool": {
            "must": [
                {"term": {"event_type": "http"}}
            ],
            "should": [
                {"wildcard": {"http.url": f"*{path}*"}} for path in suspicious_paths
            ],
            "minimum_should_match": 1
        }
    }
    
    result = es.search(index=index_name, query=query, size=100)
    
    print(f"🚨 Requêtes HTTP suspectes trouvées: {result['hits']['total']['value']}\n")
    
    for hit in result['hits']['hits']:
        source = hit['_source']
        http = source.get('http', {})
        print(f"⚠️  {source.get('src_ip', 'N/A')} → {http.get('hostname', 'N/A')}{http.get('url', 'N/A')}")
        print(f"   Méthode: {http.get('http_method', 'N/A')}, Status: {http.get('status', 'N/A')}")
        print()
    
    return result

# Exécuter la recherche
try:
    find_suspicious_http_requests()
except Exception as e:
    print(f"⚠️  Erreur: {e}")
    print("   Vérifiez que vous avez des événements HTTP dans vos logs")

🚨 Requêtes HTTP suspectes trouvées: 0



## Partie 8 : Résumé et bonnes pratiques

### Bonnes pratiques pour un SIEM

1. **Indexation optimale**
   - Utilisez des index avec rotation temporelle (ex: `suricata-YYYY.MM.DD`)
   - Configurez des index templates pour automatiser la création
   - Définissez des politiques de rétention (ILM - Index Lifecycle Management)

2. **Mapping des champs**
   - Définissez correctement les types de champs (IP, date, keyword, text)
   - Utilisez des champs `keyword` pour les agrégations
   - Utilisez des champs `text` pour la recherche full-text

3. **Sécurité**
   - Activez la sécurité Elasticsearch en production
   - Utilisez des certificats SSL/TLS
   - Configurez des rôles et permissions appropriés

4. **Performance**
   - Monitorer la santé du cluster
   - Ajuster le nombre de shards selon le volume de données
   - Utiliser des alias pour les index

5. **Alerting**
   - Créez des règles d'alerte pertinentes (pas trop, pas trop peu)
   - Testez régulièrement les alertes
   - Documentez les procédures de réponse aux incidents

6. **Maintenance**
   - Surveillez les logs Elasticsearch et Kibana
   - Effectuez des sauvegardes régulières
   - Mettez à jour régulièrement les règles Suricata

### Ressources supplémentaires

- **Documentation Suricata** : https://suricata.readthedocs.io/
- **Documentation Filebeat** : https://www.elastic.co/guide/en/beats/filebeat/current/index.html
- **Documentation Elasticsearch** : https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html
- **Documentation Kibana** : https://www.elastic.co/guide/en/kibana/current/index.html
- **Elastic Security** : https://www.elastic.co/security

### Prochaines étapes

1. Intégrer d'autres sources de logs (Apache, Nginx, Windows Event Logs, etc.)
2. Configurer des enrichissements (géolocalisation IP, threat intelligence)
3. Créer des playbooks de réponse aux incidents
4. Mettre en place une stack complète avec Logstash si nécessaire
5. Explorer Elastic Security (anciennement SIEM) pour des fonctionnalités avancées

In [19]:
# Fonction utilitaire : Nettoyage des ressources
def cleanup():
    """Supprime l'index de test (optionnel)"""
    response = input("Voulez-vous supprimer l'index 'suricata-logs'? (oui/non): ")
    if response.lower() in ['oui', 'o', 'yes', 'y']:
        if es.indices.exists(index=index_name):
            es.indices.delete(index=index_name)
            print(f"✅ Index '{index_name}' supprimé")
        else:
            print(f"ℹ️  L'index '{index_name}' n'existe pas")
    else:
        print("ℹ️  Index conservé")

# Décommenter pour nettoyer
# cleanup()

print("\n" + "=" * 60)
print("✅ TD terminé!")
print("=" * 60)
print("\n📚 N'oubliez pas de:")
print("   1. Explorer Kibana pour créer vos visualisations")
print("   2. Configurer des alertes dans Kibana")
print("   3. Tester avec de vrais logs Suricata")
print("   4. Documenter vos règles et procédures")
print("\n🎓 Bonne continuation avec votre SIEM!")


✅ TD terminé!

📚 N'oubliez pas de:
   1. Explorer Kibana pour créer vos visualisations
   2. Configurer des alertes dans Kibana
   3. Tester avec de vrais logs Suricata
   4. Documenter vos règles et procédures

🎓 Bonne continuation avec votre SIEM!
